## Untidy to Tidy Data Automation

Because this notebook extracts sensistive data from the host file, most outputs have been cleared before committing. I am only sharing my code so that all sensitive information is thoroughly excluded.

Author: Jadi Smith

In [ ]:
pip install pdfplumber

In [1]:
import pdfplumber
import pandas as pd

In [8]:
#finding the pages that i need
use_pages = []
with pdfplumber.open("data/071825D.pdf") as pdf:
   for page in pdf.pages:
       text = page.extract_text()
       if text and "N198 N257" in text:
           use_pages.append(page.page_number)

use_pages.sort()

#excluding data from the pages that i do not need
excluded_words = [
   "TOTAL CMS",
   "HEADER EOBS",
   "LNN FROM DTE-THRU DTE POS SPEC PROC CD M1 M2 M3 M4 UNITS BILLED/ALLWD BILLED ALLOWED COB PAID STATUS",
   "HEADER REMARK CODES: N198 N257",
   "------------------------------------------------------------------------------------------------------------------------------------------------------",
]

#functions to extract cleaned lines from each page
def get_cleaned_lines():
   all_lines = []
   with pdfplumber.open("data/071825D.pdf") as pdf:
       for page_num in use_pages:
           text = pdf.pages[page_num - 1].extract_text()
           if text:
               lines = text.splitlines()
               for idx, line in enumerate(lines):
                   if (idx == 9 or idx >= 13) and not any(word in line.upper() for word in excluded_words):
                       all_lines.append(line)
   return all_lines

cleaned_data = get_cleaned_lines()

#icn lines
def extract_icn(lines):
   result = {"icn": [], "member_id": [], "member_name": [], "billed_date": [], "p_auth_no": [], "patient_number": []}
   for line in lines:
       parts = line.strip().split()
       if len(parts) >= 6 and parts[0].isdigit() and len(parts[0]) == 13:
           result["icn"].append(parts[0])
           result["member_id"].append(parts[1])
           
           name_parts = []
           for part in parts[2:]:
               if part[0].isdigit():
                   break
               name_parts.append(part)
           full_name = " ".join(name_parts)
           result["member_name"].append(full_name)
           
           billed_date_index = 2 + len(name_parts) #bc the billed date is always after the 2nd idx name part
           result["billed_date"].append(parts[billed_date_index])
           
           if len(parts[-2]) == 12:
               result["p_auth_no"].append(parts[-2])
           else:
               result["p_auth_no"].append("")
               
           result["patient_number"].append(parts[-1])
   return result
    
#cos line
def extract_cos(lines):
   result = {
       "cos": [], "from_date": [], "thru_date": [], "billed": [], "allowed": [],
       "copay_deduct": [], "pt_liab": [], "cob": [], "tot_paid": [], "status": []
   }
   for line in lines:
       parts = line.strip().split()
       if len(parts) >= 10 and parts[0].isdigit() and len(parts[0]) == 3:
           result["cos"].append(parts[0])
           result["from_date"].append(parts[1])
           result["thru_date"].append(parts[2])
           result["billed"].append(parts[3])
           result["allowed"].append(parts[4])
           result["copay_deduct"].append(parts[5])
           result["pt_liab"].append(parts[6])
           result["cob"].append(parts[7])
           result["tot_paid"].append(parts[8])
           result["status"].append(parts[9])
   return result
    
# building df to export to csv
icn_df = pd.DataFrame(extract_icn(cleaned_data))
cos_df = pd.DataFrame(extract_cos(cleaned_data))

In [3]:
#npi lines
all_npi_data = {"npi":[]}
def npi_line():
   for page in use_pages:
       page_text = pdf.pages[page - 1].extract_text()
       lines = page_text.splitlines()
       #npi for the page
       current_npi = ""
       for line in lines:
           if "NPI" in line:
               parts = line.strip().split()
               for i, p in enumerate(parts):
                   if p == "NPI" and i + 1 < len(parts):
                       current_npi = parts[i + 1]
       # matching npi per icn line
       for line in lines:
           parts = line.strip().split()
           if len(parts) > 0 and len(parts[0]) == 13: 
               all_npi_data["npi"].append(current_npi)
   return all_npi_data
results = npi_line()

npi_df = pd.DataFrame(all_npi_data)

In [4]:
# lnn lines
def extract_lnn(lines):
   result = {
       "lnn": [], "from_date": [], "thru_date": [],
       "pos": [], "spec": [], "proc_cd": [], "m1": [],
       "m2": [], "m3": [], "m4": [],
       "units_billed": [], "units_allowed": [],
       "billed": [], "allowed": [], "cob": [],
       "paid": [], "status": []
   }
   found_lnn = False
   for line in lines:
       if line.startswith("1 "):
           found_lnn = True
           parts = line.split()
           # starting columns
           result["lnn"].append(parts[0])
           result["from_date"].append(parts[1])
           result["thru_date"].append(parts[2])
           result["pos"].append(parts[3])
           result["spec"].append(parts[4])
           result["proc_cd"].append(parts[5])
           result["m1"].append(parts[6])
           # m1-4 exist
           if all(len(p) == 2 for p in parts[7:10]):
               result["m2"].append(parts[7])
               result["m3"].append(parts[8])
               result["m4"].append(parts[9])
               result["units_billed"].append(parts[10])
               result["units_allowed"].append(parts[11])
               result["billed"].append(parts[12])
               result["allowed"].append(parts[13])
           # m1-3 exist
           elif all(len(p) == 2 for p in parts[7:9]) and len(parts[9]) != 2:
               result["m2"].append(parts[7])
               result["m3"].append(parts[8])
               result["m4"].append("")
               result["units_billed"].append(parts[9])
               result["units_allowed"].append(parts[10])
               result["billed"].append(parts[11])
               result["allowed"].append(parts[12])
           # m1-2 exist
           elif len(parts[7]) == 2 and len(parts[8]) != 2:
               result["m2"].append(parts[7])
               result["m3"].append("")
               result["m4"].append("")
               result["units_billed"].append(parts[8])
               result["units_allowed"].append(parts[9])
               result["billed"].append(parts[10])
               result["allowed"].append(parts[11])
           # m1 only exists
           else:
               result["m2"].append("")
               result["m3"].append("")
               result["m4"].append("")
               result["units_billed"].append(parts[7])
               result["units_allowed"].append(parts[8])
               result["billed"].append(parts[9])
               result["allowed"].append(parts[10])
           # always last three columns
           result["cob"].append(parts[-3])
           result["paid"].append(parts[-2])
           result["status"].append(parts[-1])
   # if no lnn line at all, add one empty row
   if not found_lnn:
       for key in result:
           result[key].append("")
   return result
lnn_df = pd.DataFrame(extract_lnn(cleaned_data))

In [5]:
lnn_df.to_csv("lnn.csv", index=False)

In [6]:
final_df = pd.concat([npi_df,lnn_df,icn_df, cos_df], axis=1)
final_df.to_csv("cleaned_billing_pdf.csv", index=False)

In [7]:
final_df = pd.concat([icn_df, cos_df], axis=1)